In [1]:
#Step 1: load data.

In [2]:
#Step 1a: load data from ACS 
#using censusdata package and documentation 
#from here: https://github.com/jtleider/censusdata

In [3]:
import sys
sys.path.append('..')

In [4]:
from src.data.censusdata import censusdata

In [5]:
import pandas as pd

In [6]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [7]:
#Search for table and variable combinations for desired stats.

In [8]:
acs_version = 'acs5' #or 'acs5'

In [9]:
#search for race-related variables
#output supressed.

#censusdata.search(acs_version, 2018, 'label', 'white')

In [10]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B02001'))

In [11]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B03002'))

In [12]:
#White, not Hispanic: B03002_003E (from total: B03002_001E)
#White, Hispanic: B03002_013E (from total: B03002_001E)
#Black: B02001_003E (from total: B02001_001E)
#Other: If totals are equal: (B02001_001E and B03002_001E) then other is total minus 
#White, not Hispanic plus White, Hispanic plus Black.

#Will check it out after finding geography codes.

In [13]:
#search for Socioeconomic status-related variables
#censusdata.search(acs_version, 2018, 'label', 'degree')

In [14]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B15003'))

In [15]:
#censusdata.search(acs_version, 2018, 'label', 'per capita income')

In [16]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B19301'))

In [17]:
#Finding geography codes for all counties in Texas

In [18]:
#output supressed.
#censusdata.geographies(censusdata.censusgeo([('state', '*')]), acs_version, 2018)
#Texas is '48'
#censusdata.geographies(censusdata.censusgeo([('state', '48'), ('county', '*')]), acs_version, 2018)
#Travis County is '453' (used for testing)

In [19]:
txcounties = censusdata.download(acs_version, 2018,
                             censusdata.censusgeo([('state', '48'), ('county', '*')]),
                             ['B03002_001E','B02001_001E','B03002_003E','B03002_013E','B02001_003E', \
                              'B15003_022E','B15003_001E', \
                              'B19301_001E'])


In [20]:
#Check that total from the two tables are the same for all counties in Texas.
assert (txcounties.B03002_001E == txcounties.B02001_001E).all()

In [21]:
txcounties['percent_white_nh'] = txcounties.B03002_003E / txcounties.B03002_001E * 100
txcounties['percent_hispanic'] = txcounties.B03002_013E / txcounties.B03002_001E * 100
txcounties['percent_black'] = txcounties.B02001_003E / txcounties.B02001_001E * 100
txcounties['percent_other'] = 100. - txcounties['percent_white_nh'] - \
                                txcounties['percent_hispanic'] - txcounties['percent_black']

txcounties['percent_w4yrdeg'] = txcounties.B15003_022E / txcounties.B15003_001E * 100
txcounties['per_capita_income'] = txcounties.B19301_001E


In [22]:
txcounties = txcounties[['percent_white_nh', 'percent_hispanic', 'percent_black','percent_other', \
                         'percent_w4yrdeg','per_capita_income']]
txcounties.describe()

,percent_white_nh,percent_hispanic,percent_black,percent_other,percent_w4yrdeg,per_capita_income
count,254.00,254.00,254.00,254.00,254.00,254.00
mean,56.13,27.77,6.34,9.76,12.88,25497.25
std,21.21,21.54,6.61,8.06,4.77,5292.63
min,0.73,1.68,0.00,0.29,0.00,13350.00
25%,41.94,13.35,1.07,4.80,9.85,22265.50
50%,58.80,19.55,4.13,7.46,12.06,25128.50
75%,73.10,37.87,9.12,12.03,14.60,28453.75
max,90.66,94.35,33.84,58.29,33.05,43439.00
